In [4]:
# Install the OpenAI Library
!pip install openai==0.28

   ---------------------------------------- 0.0/76.5 kB ? eta -:--:--
   ---------------------------------------- 76.5/76.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.51.2
    Uninstalling openai-1.51.2:
      Successfully uninstalled openai-1.51.2


In [1]:
!pip install python-dotenv


In [6]:
import openai
import os
from dotenv import load_dotenv
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load your OpenAI API key from the .env file
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Example DataFrame of travel destinations
data = {
    'destination': ['Paris', 'Tokyo', 'Bangkok', 'New York', 'Sydney'],
    'interests': ['culture, art, history', 'technology, anime, food', 'beaches, nightlife, temples',
                  'shopping, food, landmarks', 'beaches, adventure, wildlife'],
    'budget_range': ['high', 'high', 'medium', 'medium', 'low'],
    'travel_type': ['relaxation', 'adventure', 'adventure', 'adventure', 'relaxation']
}
df_destinations = pd.DataFrame(data)

# Function to gather user preferences and print them
def gather_preferences():
    print("Welcome to AI-Powered Travel Advisor!")
    
    # Collect user inputs
    interests = input("Enter your interests (e.g., culture, beaches, nightlife): ")
    budget = input("Enter your budget (low, medium, high): ")
    travel_type = input("Enter preferred travel type (e.g., adventure, relaxation): ")
    
    # Print user inputs for each step
    print("\nYou provided the following details:")
    print(f"Interests: {interests}")
    print(f"Budget: {budget}")
    print(f"Travel Type: {travel_type}")
    
    return interests, budget, travel_type

# Function to recommend destinations
def recommend_destinations(interests, budget, travel_type):
    # Vectorize the 'interests' column to measure similarity
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df_destinations['interests'])

    # Calculate similarity based on user input
    user_input = [interests]
    user_vec = vectorizer.transform(user_input)
    cosine_sim = cosine_similarity(user_vec, X).flatten()

    # Filter destinations by budget and travel type
    df_filtered = df_destinations[(df_destinations['budget_range'] == budget) & 
                                  (df_destinations['travel_type'] == travel_type)].copy()

    # Apply cosine similarity to the filtered DataFrame by aligning indices properly
    df_filtered['similarity'] = cosine_sim[df_filtered.index]
    
    # Sort by similarity score and reset the index to ensure proper assignment
    df_filtered = df_filtered.sort_values(by='similarity', ascending=False).reset_index(drop=True)

    # Return top recommendations
    return df_filtered[['destination', 'similarity']]

# Function to ask OpenAI API questions
def ask_openai(question):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=question,
        max_tokens=100
    )
    return response.choices[0].text.strip()

# Main interactive function
def travel_advisor():
    interests, budget, travel_type = gather_preferences()
    
    # Get recommendations based on user preferences
    recommendations = recommend_destinations(interests, budget, travel_type)
    
    if not recommendations.empty:
        print("\nTop destination recommendations based on your preferences:\n")
        print(recommendations[['destination', 'similarity']])

        # Ask if user wants to know more about the destinations
        more_info = input("\nDo you want to ask questions about these destinations? (yes/no): ").lower()
        
        while more_info == 'yes':
            question = input("Ask a question about one of the destinations: ")
            answer = ask_openai(question)
            print(f"\nAnswer: {answer}")
            more_info = input("\nDo you have more questions? (yes/no): ").lower()
    else:
        print("Sorry, no recommendations match your preferences.")

# Run the travel advisor
travel_advisor()


Welcome to AI-Powered Travel Advisor!

You provided the following details:
Interests: culture
Budget: medium
Travel Type: relaxation
Sorry, no recommendations match your preferences.
